### Loading the Grammar

In [2]:
import nltk
from nltk.grammar import CFG as CFG
from nltk.tree import Tree as Tree
import numpy as np

grammar = nltk.data.load('file:///' + os.path.abspath('atis-grammar-cnf.cfg')) # load the grammar
s = nltk.data.load('file:///' + os.path.abspath('atis-test-sentences.txt')) # load raw sentences
t = nltk.parse.util.extract_test_sentences(s) # extract test sentences 

LookupError: 
**********************************************************************
  Resource [93mg:[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('g:')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93m/g:/My Drive/UdS/Classes/Computational Linguistics/a3/Submission/atis-grammar-cnf.cfg[0m

  Searched in:
    - ''
**********************************************************************


In [62]:
class BuffRule:
    def __init__(self, A, B, Bij=None, C=None, Cij=None): # 'None' arguments let this work for leaves as well as nodes
        """
        Back pointers are confusing. Don't let them get you down!
        
        ex: production rule: HMC -> NOUN_NN HMD
        becomes: production() = BuffRule(   A=str("HMC"),
                                            B=str("NOUN_NN"),
                                            Bij=tuple(DataFram[i], DataFram[j]),
                                            C=str("HMD"),
                                            Cij=tuple(DataFram[i], DataFram[j]))

        So, when building your trees, it's as simple as looking up production rule A, looking into Bij and Cij, and checking B/C.
        """
        self.A = A  # Rule LHS
        self.B = B  # Rule RHS1
        self.Bij = Bij # Rule RHS1 index
        self.C = C # Rule RHS2
        self.Cij = Cij # Rule RHS2 index

    def __str__(self):
        return self.A # for checking the "name" of this rule in a cell, all we need is the LHS A

In [87]:
"""
Data structure: Ch(i,k) eventually contains {A | A ⇒* wi ... wk-1} (initially empty)

for each i from 1 to n:
    for each production rule A → wi:
        add A to Ch(i, i+1) 

for each width b from 2 to n:
    for each start position i from 1 to n-b+1:
        for each left width k from 1 to b-1:
            for each B ∈ Ch(i, i+k) and C ∈ Ch(i+k,i+b):
                for each production rule A → B C:
                    add A to Ch(i,i+b) claim that w ∈ L(G) iff S ∈ Ch(1,n+1)
"""

def parse(sentence):
    if type(sentence) == str:
        sentence = sentence.split()
    n = len(sentence) # n | n = word length
    Ch = np.empty((n,n), dtype=set) # nxn array of sets, array[row[list[parent node(s)]]]
    buffCh = np.empty((n,n), dtype=set) # nxn array of sets of nodes(BuffRule objects), array[row[list[BuffRules(s)]]]
    for i in range(n):
        for j in range(n):
            Ch[i][j] = set() # there must be a way to populate with empty sets, I just haven't found it.
            buffCh[i][j] = set()
    for j in range(n): # for each i from 1 to n:
        for rule in grammar.productions(rhs=sentence[j]):
            Ch[j][j].add(rule.lhs()) # for each production rule A → wi: add A to Ch(i, i+1)
            buffCh[j][j].add(BuffRule(A=rule.lhs(), B=sentence[j]))
    for j in range(n):
        for i in range(j-1, -1, -1): # I hate these stepwise increments, especially since Koller does it reversed form the book,
            for k in range(i, j):    #  and neither worked in practice, so I had to make my own loop index rules.
                for rule in grammar.productions():
                    if rule.rhs()[0] in Ch[i][k] and rule.rhs()[1] in Ch[k+1][j]:
                        Ch[i][j].add(rule.lhs()) # rule grid for grammaticality check
                        buffCh[i][j].add(BuffRule(A=rule.lhs(), B=rule.rhs()[0], Bij=(i,k), C=rule.rhs()[1], Cij=(k+1,j))) # BuffRule grid for building trees.
    return buffCh

In [64]:
def draw_tree(node):
    if node.C == None:
        return f"({node.A}: {node.B})" # base case, print the leaf and its production rule
    else:
        for item in grid[node.Bij[0]][node.Bij[1]]: # checking rules in the cell indicated by backpointer for B
            if item.A == node.B:
                new_node_B = item
        for item in grid[node.Cij[0]][node.Cij[1]]: # checking rules in the cell indicated by backpointer for C
            if item.A == node.C:
                new_node_C = item
        return f"({node.A} ({draw_tree(new_node_B)}  {draw_tree(new_node_C)}))" # recursively building next branch/nodes in tree

### Parsing and drawing ATIS test sentences...

    Used to take about 12s to run the recogniser, but now looking at counts it's suddenly taking several minutes?
    Most recent run was 7m41s
    Sorry :-\
    Refactored as much as I could, but coulnd't recreate the 12s checks. Dunno.

In [92]:
results = []
#trees = []
# i = 0
for sentence in t:
    grid = parse(sentence[0]) #  >:(
    count = 0
    for item in grid[0][-1]: 
        if str(item.A) == 'SIGMA': # claim that w ∈ L(G) iff S ∈ Ch(1,n+1)
            # trees.append(draw_tree(item)) # glorious, wonderful trees!
            count += 1
    results.append((sentence[0], count))

In [93]:
# twee = Tree.fromstring(trees[0])      ### Turn this section on for printing tree samples
# print(twee.leaves())
# twee_weaves = twee.leaves()
# print(" ".join(twee_weaves))
# for tree in trees:
#     print(Tree.fromstring(tree))

with open("output.txt", 'w', encoding="utf-8") as file_out:
    for result in results:
        file_out.write(str(str(" ".join(result[0])) + "\t" + str(result[1]) + " parses\n"))